# ECS7020P mini-project submission

The mini-project has two separate components:


1.   **Basic component** [6 marks]: Using the MLEnd London Sounds dataset, build a machine learning pipeline that takes as an input an audio segment and predicts whether the audio segment has been recorded indoors or outdoors.
2.   **Advanced component** [10 marks]: Formulate your own machine learning problem and build a machine learning solution using the MLEnd London Sounds dataset. 

Your submission will consist of two Jupyter notebooks, one for the basic component and another one for advanced component. Please **name each notebook**:

* ECS7020P_miniproject_basic.ipynb
* ECS7020P_miniproject_advanced.ipynb

then **zip and submit them toghether**.

Each uploaded notebook should include: 

*   **Text cells**, describing concisely each step and results.
*   **Code cells**, implementing each step.
*   **Output cells**, i.e. the output from each code cell.

and **should have the structure** indicated below. Notebooks might not be run, please make sure that the output cells are saved.

How will we evaluate your submission?

*   Conciseness in your writing (10%).
*   Correctness in your methodology (30%).
*   Correctness in your analysis and conclusions (30%).
*   Completeness (10%).
*   Originality (10%).
*   Efforts to try something new (10%).

Suggestion: Why don't you use **GitHub** to manage your project? GitHub can be used as a presentation card that showcases what you have done and gives evidence of your data science skills, knowledge and experience. 

Each notebook should be structured into the following 9 sections:


# 1 Author

**Student Name**:  
**Student ID**:  



# 2 Problem formulation

Describe the machine learning problem that you want to solve and explain what's interesting about it.

# 3 Machine Learning pipeline

Describe your ML pipeline. Clearly identify its input and output, any intermediate stages (for instance, transformation -> models), and intermediate data moving from one stage to the next. It's up to you to decide which stages to include in your pipeline. 

# 4 Transformation stage

Describe any transformations, such as feature extraction. Identify input and output. Explain why you have chosen this transformation stage.

# 5 Modelling

Describe the ML model(s) that you will build. Explain why you have chosen them.

# 6 Methodology

Describe how you will train and validate your models, how model performance is assesssed (i.e. accuracy, confusion matrix, etc)

# 7 Dataset

Describe the dataset that you will use to create your models and validate them. If you need to preprocess it, do it here. Include visualisations too. You can visualise raw data samples or extracted features.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa.display
import math

import os, sys, re, pickle, glob
import urllib.request
import zipfile

import IPython.display as ipd
from tqdm import tqdm
import librosa
import scipy.stats as stats

In [3]:
sample_path = 'MLEndLS/*.wav'
files = glob.glob(sample_path)
MLENDLS_df = pd.read_csv('./MLEndLS.csv').set_index('file_id') 

In [33]:
import matplotlib.pyplot as plt

aud_data = []
labels = []

def getPitch(x,fs,winLen=0.02):
    #winLen = 0.02 
    p = winLen*fs
    frame_length = int(2**int(p-1).bit_length())
    hop_length = frame_length//2
    f0 = librosa.yin(y=x, fmin=300, fmax=1000, sr=fs,
                frame_length=frame_length,hop_length=hop_length)
    return f0

for n in range(0,len(files)):
# for n in range(15,150):
    a, fs = librosa.load(files[n],sr=None)    
    
    S = np.abs(librosa.stft(a))

    # Trimming to get a standard input size and reduce frquency range (after visual inspection of spectogram)
    S = S[10:100,:536]
    S = S.reshape(1,S.shape[0]*S.shape[1])[0]

    # Filtering errored/invalid data
    if len(S) < 90*536 : continue
    if (n%100 == 0): print(n)
    
    aud_data.append(S)
    labels.append(MLENDLS_df.loc[files[n].split('/')[-1]].in_out)
    
print("Done")

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
Done


In [34]:
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

numComponents = 5
pca = PCA(n_components=numComponents)
pca.fit(aud_data)

projected = pca.transform(aud_data)
projected = pd.DataFrame(projected,columns=['pc1','pc2','pc3','pc4','pc5'],index=range(1,len(aud_data)+1))
# projected['label'] = labels
display(projected)

,pc1,pc2,pc3,pc4,pc5
1,-225.875466,-30.413585,-24.799274,-98.034485,23.855877
2,-109.298090,-133.198104,17.015800,27.937171,40.418684
3,-246.337859,27.005723,-12.768900,51.206973,50.036350
4,-213.374926,69.485625,187.107505,-108.240533,-117.322421
5,-300.415214,-2.594630,-1.983354,-20.964679,8.282350
...,...,...,...,...,...
2475,-277.373929,14.298572,-38.487936,-36.465191,64.643020
2476,106.790441,-46.195578,-81.706900,-39.576311,72.463081
2477,-177.154952,-23.065343,-53.508946,1.938133,68.998650
2478,-80.723152,-25.414604,-73.530950,-160.310314,85.243118


In [39]:
projected = projected.drop(columns=[
#     'pc3',
    'pc4',
    'pc5',
#     'pc6',
#     'pc7'
])

In [40]:
from sklearn.model_selection import GridSearchCV, train_test_split

X_train, X_val, y_train, y_val = train_test_split(projected,labels,test_size=0.2)
print(X_train.shape, X_val.shape)

(1983, 3) (496, 3)


# 8 Results

Carry out your experiments here, explain your results.

In [41]:
from sklearn import svm

parameters = {'C':[1,2,3,4,5]}

svc = svm.SVC()
clf = GridSearchCV(svc, parameters,cv=5)

clf.fit(X_train,y_train)

print('Hyperparameters: ', clf.best_estimator_)
print('Average accuracy: ', clf.best_score_)
print('Test dataset accuracy:', clf.score(X_val, y_val))

Hyperparameters:  SVC(C=4)
Average accuracy:  0.6354056942218151
Test dataset accuracy: 0.657258064516129


In [42]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

from sklearn.model_selection import GridSearchCV, train_test_split

X_train, X_val, y_train, y_val = train_test_split(projected,labels,test_size=0.2)
print(X_train.shape, X_val.shape)

clf = RandomForestClassifier(max_depth=7)
clf.fit(X_train,y_train)

yt_p = clf.predict(X_train)
yv_p = clf.predict(X_val)

print('Training Accuracy', np.mean(yt_p==y_train))
print('Validation  Accuracy', np.mean(yv_p==y_val))

(1983, 3) (496, 3)
Training Accuracy 0.7155824508320726
Validation  Accuracy 0.6592741935483871


In [43]:
parameters = {'n_estimators':[50,200,300,500], 'max_features':[0.01,0.05,1], 'max_depth':[7,8]}

rfc = RandomForestClassifier(max_depth=7)
clf = GridSearchCV(rfc, parameters,cv=5)

clf.fit(X_train,y_train)

print('Hyperparameters: ', clf.best_estimator_)
print('Average accuracy: ', clf.best_score_)
print('Test dataset accuracy:', clf.score(X_val, y_val))

Hyperparameters:  RandomForestClassifier(max_depth=7, max_features=1, n_estimators=200)
Average accuracy:  0.6298310561534742
Test dataset accuracy: 0.6653225806451613


In [267]:
model  = svm.SVC(C=1)
model.fit(X_train,y_train)

yt_p = model.predict(X_train)
yv_p = model.predict(X_val)

print('Training Accuracy', np.mean(yt_p==y_train))
print('Validation  Accuracy', np.mean(yv_p==y_val))
print('The support vectors are', model.support_vectors_.shape)

Training Accuracy 0.7594553706505295
Validation  Accuracy 0.7358870967741935
The support vectors are (1202, 5)


In [100]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

clf = DecisionTreeClassifier(max_depth=None, min_samples_split=102,
     random_state=0)
scores = cross_val_score(clf, X_train, y_train, cv=5)
print(scores.mean())

clf = RandomForestClassifier(n_estimators=55, max_depth=None,
     min_samples_split=2, random_state=0)
scores = cross_val_score(clf, X_train, y_train, cv=5)
print(scores.mean())

0.5244675978932907
0.5194272701829377


# 9 Conclusions

Your conclusions, improvements, etc should go here